In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, Conversation, pipeline
import torch

In [2]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
#name = "RomanOrac/llama-2-7b-slovenian"
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.pad_token_id = tokenizer.eos_token_id    # for open-ended generation

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
tokenizer.chat_template = (
            "{% if messages[0]['role'] == 'system' %}"
            "{% set loop_messages = messages[1:] %}"
            "{% set system_message = messages[0]['content'] %}"
            "{% else %}"
            "{% set loop_messages = messages %}"
            "{% set system_message = false %}"
            "{% endif %}"
            "{% for message in loop_messages %}"
            "{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}"
            "{{ raise_exception('Conversation roles must alternate "
            "user/assistant/user/assistant/...') }}"
            "{% endif %}"
            "{% if loop.index0 == 0 and system_message != false %}"
            "{% set content = '<<SYS>>\n' + system_message + '\n<</SYS>>\n\n' + "
            "message['content'] %}"
            "{% else %}"
            "{% set content = message['content'] %}"
            "{% endif %}"
            "{% if message['role'] == 'user' %}"
            "{{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}"
            "{% elif message['role'] == 'assistant' %}"
            "{{ ' '  + content.strip() + ' ' + eos_token }}"
            "{% endif %}"
            "{% endfor %}"
            )

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

In [8]:
model.generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.7,
    #top_k=
)

In [9]:
generation_pipe = pipeline(
    "conversational",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",    # finds GPU
)

In [10]:
violence_df = pd.read_csv("violence.csv")

In [11]:
violence_df.Sentence.loc[0]

'Ker sicer nas boste prisilili, da bomo posegli s svoje rezervne višine, toda ne v volilni boj. ampak v vaša ušesa ter vam jih navili.'

In [12]:
examples_df = violence_df[violence_df["Example"] == True]

In [13]:
examples_df

,Date,Sentence,Violent,Description,Example
9,21. 11. 1925.,Princip naš bodi: odkrit in viteški boj s pošt...,True,Implied violence or violent methods,True
15,07. 02. 1925.,"Boj, ki smo ga začeli, pa bomo vodili dalje: Z...",True,Implied violence or violent methods,True
23,03. 09. 1925.,"Pripraviti vse, da bo to jutro čimpreje napoči...",True,Implied violence or violent methods,True
28,14. 02. 1925.,O tem si moramo biti vsi Orjunaši na jasnem da...,True,Implied violence or violent methods,True
33,24. 12. 1925.,"Zbrisal ga bo, kakor besen orkan z zemlje, tak...",True,Implied violence or violent methods,True
60,05. 01. 1926.,"Borba je pred nami in kri za nami, kri ki nas ...",True,Threat against other political parties,True


In [14]:
text_df = pd.DataFrame()

In [15]:
ex_violence = violence_df[violence_df["Example"] == False]
sample_val = ex_violence.sample(5)

In [16]:
system_prompt = "You are an expert classifier in Slovenian sociology. You always answer in English."

In [17]:
prompt_base_few_shot = "## Label this Slovenian text, according to wether the whole text is violent, includes hatespeech, calls to violent or subversive actions or sublevation (both explictly or figuratively), or not, indicating 'True' if it does and 'False' if it doesn't:\n"

In [18]:
FEW_SHOT = prompt_base_few_shot+"# Text: {text}\n# Violent: {label}\n# Description: {description}\n\n"

In [20]:
prompt_base = "## Label ONLY THE LAST Slovenian text, according to wether the whole text is violent, includes hatespeech, calls to violent or subversive actions or sublevation (both explictly or figuratively), or not, indicating 'True' if it does and 'False' if it doesn't:\n"

In [21]:
COT = f"""{prompt_base}
## Answer:
Let's think step by step to label this text according to wether the whole text is violent, includes hatespeech, calls to violent or subversive actions or sublevation (both explictly or figuratively), or not.

Step 1: Recognize words or locutions related to violence or calls to violent actions
- Identify key words or locutions related to violence or calls to violent action such as 'bitka', 'upor', 'strmoglavljenje vlade', or 'boj'.

Step 2: Recognize words or locutions related to hatespeech
- Identify key words or locutions related to hatespeech such as 'strupeni Žid', 'tujci ven', 'Čefur', or 'kurbe'.

Step 3: Label the text based on the previous steps
- If you do find instances of violence, calls to violent actions, or hatespeech, label the text as 'Violent: True'
- If you do not find any instances of violence, calls to violent actions, or hatespeech, label the text as 'Violent: False'
- Include a description of why the text has been labeled as such
"""

In [22]:
text_df["prompts"] = examples_df[["Sentence", "Violent", "Description"]].astype(str).apply(lambda x: FEW_SHOT.format(text=x['Sentence'],label=x['Violent'],description=x['Description']), axis=1)

In [23]:
few_shots = '## I present you here six (6) examples of correctly labeled texts based on similar problems:\n' + "\n".join(text_df["prompts"].to_list()) + "\n\nUse the previous examples as guidelines without labelling them.\n\n"

In [24]:
def label(texts):
    conversation = Conversation()
    conversation.add_message({'role':'system', 'content': system_prompt})
    conversation.add_message({'role':'user', 'content': few_shots+COT+prompt_base+"# Text: " + texts + "\n# Let's think step by step: "})
    response = generation_pipe(
        conversation,
        max_new_tokens=1024,
        eos_token_id=tokenizer.eos_token_id,
    )
    return response

In [25]:
sample_val

,Date,Sentence,Violent,Description,Example
47,13. 11. 1926.,Orjuna je razpuščena ali ostali so še vedno Or...,True,Implied violence or violent methods,False
43,13. 02. 1926.,Nadaljevali bomo našo redukcijo dobrih iin poš...,True,Implied violence or violent methods,False
96,17. 04. 1926.,"Ostali smo, kot večen memento bližnjega obraču...",True,Implied violence or violent methods,False
53,08. 05. 1926.,Orjuna vas svari pred nadaljevanjem teh eksper...,True,Implied violence or violent methods,False
82,07. 08. 1926.,Bil pa ie tudi znam enje popolne zm age Orjune...,True,Implied violence or violent methods,False


In [26]:
label(ex_violence.Sentence[65])

Conversation id: 8a431884-8225-4964-9cb1-20a46f45e2a0
system: You are an expert classifier in Slovenian sociology. You always answer in English.
user: ## I present you here six (6) examples of correctly labeled texts based on similar problems:
## Label this Slovenian text, according to wether the whole text is violent, includes hatespeech, calls to violent or subversive actions or sublevation (both explictly or figuratively), or not, indicating 'True' if it does and 'False' if it doesn't:
# Text: Princip naš bodi: odkrit in viteški boj s poštenimi sredstvi, ki morajo služiti vedno samo naciji i ideji!
# Violent: True
# Description: Implied violence or violent methods


## Label this Slovenian text, according to wether the whole text is violent, includes hatespeech, calls to violent or subversive actions or sublevation (both explictly or figuratively), or not, indicating 'True' if it does and 'False' if it doesn't:
# Text: Boj, ki smo ga začeli, pa bomo vodili dalje: Za Nacijo — do Pobe